In [ ]:
# ต้องมาก่อนการใช้ TensorFlow ทุกอย่าง!
import os
import random
import numpy as np
import tensorflow as tf
from typing import Tuple, List
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import warnings
from river.drift import ADWIN  # นำเข้า ADWIN จากไลบรารี river

# ===== ตั้งค่าความเสถียรและ reproducibility =====
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# ปิด multi-threading ของ TensorFlow
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

warnings.filterwarnings('ignore')

# --- 1. Data Preparation and Feature Engineering ---
try:
    df = pd.read_csv("nvidia_10yr_data.csv", parse_dates=["Date"])
except FileNotFoundError:
    print("Error: 'nvidia_10yr_data.csv' not found. Please ensure the file is in the same directory.")
    exit()

df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
df = df.sort_values("Date")

# Feature engineering
df['Return'] = df['Close'].pct_change()
df['Volatility'] = df['Close'].rolling(7).std()
#df['Price_Diff'] = df['High'] - df['Low']
df['Volume_Log'] = np.log1p(df['Volume'])
# Feature interaction: Return * Volume_Log
df['Return_Volume'] = df['Return'] * df['Volume_Log']
#df['momentum_10'] = df['Close'] - df['Close'].shift(10)

# Drop NaN after rolling calculations
df.dropna(inplace=True)

#X = df[['Return', 'Volatility', 'Volume_Log','Return_Volume']]
X = df[['Return', 'Volatility','Volume_Log', 'Return_Volume']]
#X = df[['Return','Volume_Log','Return_Volume']]
y = df['Close']


# --- 2. Model and Data Utility Classes ---
class SequenceGenerator:
    """Class to create sequence data for RNN-based models and handle scaling."""
    def __init__(self, sequence_length: int = 30):
        self.sequence_length = sequence_length
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()
        
    def create_sequences(self, X: pd.DataFrame, y: pd.Series, fit_scalers: bool = True):
        X_scaled = self.scaler_X.fit_transform(X) if fit_scalers else self.scaler_X.transform(X)
        y_scaled = self.scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() if fit_scalers else self.scaler_y.transform(y.values.reshape(-1, 1)).flatten()
        
        X_seq, y_seq = [], []
        for i in range(self.sequence_length, len(X_scaled)):
            X_seq.append(X_scaled[i-self.sequence_length:i])
            y_seq.append(y_scaled[i])
        
        return np.array(X_seq), np.array(y_seq)
    
    def inverse_transform_y(self, y_scaled):
        return self.scaler_y.inverse_transform(y_scaled.reshape(-1, 1)).flatten()

class RNNRegressor:
    """Universal RNN Regressor supporting SimpleRNN, LSTM, and GRU models."""
    def __init__(self, model_type: str = 'LSTM', sequence_length: int = 30, units: int = 128, dropout_rate: float = 0.3, learning_rate: float = 0.0005, epochs: int = 200, batch_size: int = 32, verbose: int = 0):
        self.model_type = model_type.upper()
        self.sequence_length = sequence_length
        self.units = units
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
        self.seq_generator = SequenceGenerator(sequence_length)
        if self.model_type not in ['RNN', 'LSTM', 'GRU']:
            raise ValueError("model_type must be 'RNN', 'LSTM', or 'GRU'")
        
    def _get_layer_type(self):
        if self.model_type == 'RNN': return SimpleRNN
        elif self.model_type == 'LSTM': return LSTM
        elif self.model_type == 'GRU': return GRU
        
    def _build_model(self, input_shape):
        LayerType = self._get_layer_type()
        model = Sequential([
            LayerType(self.units, return_sequences=True, input_shape=input_shape),
            Dropout(self.dropout_rate),
            LayerType(self.units // 2, return_sequences=True), # เพิ่มเลเยอร์
            Dropout(self.dropout_rate),
            LayerType(self.units // 4),
            Dropout(self.dropout_rate),
            Dense(16, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse', metrics=['mae'])
        return model
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        X_seq, y_seq = self.seq_generator.create_sequences(X, y, fit_scalers=True)
        if len(X_seq) == 0: raise ValueError("Not enough data to create sequences")
        self.model = self._build_model((X_seq.shape[1], X_seq.shape[2]))
        early_stopping = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True) # เพิ่ม patience
        self.model.fit(X_seq, y_seq, epochs=self.epochs, batch_size=self.batch_size, callbacks=[early_stopping], verbose=self.verbose)
        return self
    
    def predict(self, X: pd.DataFrame):
        if self.model is None: raise ValueError("Model not fitted yet")
        # Ensure X is correctly transformed
        X_scaled = self.seq_generator.scaler_X.transform(X)
        X_seq, _ = self.seq_generator.create_sequences(X, pd.Series([0] * len(X)), fit_scalers=False)
        if len(X_seq) == 0: return np.array([])
        y_pred_scaled = self.model.predict(X_seq, verbose=0)
        y_pred = self.seq_generator.inverse_transform_y(y_pred_scaled)
        return y_pred

class LinearRegressionModel:
    """Simple linear regression wrapper for compatibility."""
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.model = LinearRegression(fit_intercept=fit_intercept)
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()
        self.is_fitted = False

    def fit(self, X: pd.DataFrame, y: pd.Series):
        X_scaled = self.scaler_X.fit_transform(X)
        y_scaled = self.scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
        self.model.fit(X_scaled, y_scaled)
        self.is_fitted = True

    def predict(self, X: pd.DataFrame):
        if not self.is_fitted:
            raise ValueError("Model not fitted yet")
        X_scaled = self.scaler_X.transform(X)
        y_pred_scaled = self.model.predict(X_scaled)
        y_pred = self.scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
        return y_pred

# --- 3. Concept Drift Detection (ใช้ ADWIN แทน) ---
class ADWINDriftDetector:
    """Detects concept drift points using the ADWIN algorithm.
    จะนับเฉพาะ drift ที่มีข้อมูลพอ และ merge fold สุดท้ายหากข้อมูลไม่พอ"""

    def __init__(self, delta: float = 0.002, min_fold_len: int = 60):
        self.detector = ADWIN(delta=delta)
        self.min_fold_len = min_fold_len
        self.drift_points_: List[int] = []

    def detect(self, data: pd.DataFrame, target_column: str, seq_len: int = 30, test_ratio: float = 0.2) -> List[int]:
        self.drift_points_ = []
        series_to_monitor = data[target_column]
        drift_points_temp = []
        last_point = 0

        # Step 1: Detect drift points
        for i, val in enumerate(series_to_monitor):
            self.detector.update(val)
            if self.detector.drift_detected:
                fold_len = i - last_point
                split_idx = int((1-test_ratio)*fold_len)
                train_len = split_idx
                test_len = fold_len - split_idx
                if fold_len >= self.min_fold_len and train_len >= seq_len and test_len >= seq_len:
                    drift_points_temp.append(i)
                    last_point = i
                # ถ้าไม่พอข้อมูล ให้ข้าม drift นี้ไปเลย

        # Step 2: พิจารณา fold สุดท้าย (ช่วง drift สุดท้าย -> len(data))
        all_points = [0] + drift_points_temp + [len(data)]
        last_drift_idx = len(drift_points_temp) - 1
        prev_drift = drift_points_temp[last_drift_idx] if last_drift_idx >= 0 else 0
        final_fold_len = len(data) - prev_drift
        split_idx = int((1-test_ratio)*final_fold_len)
        train_len = split_idx
        test_len = final_fold_len - split_idx

        # ถ้า fold สุดท้าย "ไม่พอข้อมูล" ให้ merge กับ drift ก่อนหน้า
        if final_fold_len < self.min_fold_len or train_len < seq_len or test_len < seq_len:
            # ลบ drift สุดท้ายออก (ถ้ามีมากกว่า 1 drift)
            if len(drift_points_temp) > 0:
                drift_points_temp = drift_points_temp[:-1]

        self.drift_points_ = drift_points_temp
        return self.drift_points_

# --- 4. Cross-Validation Strategies ---

class DriftAdaptiveTimeSeriesCV:
    """Performs cross-validation using a rolling window approach based on detected drift points.
    แก้ไขให้ข้ามจุด drift ที่แบ่ง train/test ไม่ได้"""
    def __init__(self, model_type: str = 'LSTM', model_params: dict = None):
        self.model_type = model_type.upper()
        self.model_params = model_params or {}

    def run(self, X: pd.DataFrame, y: pd.Series, drift_points: List[int]) -> Tuple[List[float], List[float]]:
        metrics_rmse, metrics_mae = [], []
        seq_len = self.model_params.get('sequence_length', 30)
        min_fold_len = max(seq_len * 2, 40)
        test_ratio = 0.2

        all_points = sorted(list(set([0] + drift_points + [len(X)])))
        for i in range(len(all_points) - 1):
            start = all_points[i]
            end = all_points[i+1]
            fold_length = end - start
            split_point = int(fold_length * (1 - test_ratio))
            train_start = start
            train_end = start + split_point
            test_start = train_end
            test_end = end

            train_len = train_end - train_start
            test_len = test_end - test_start

            # เงื่อนไขสำหรับ Linear ไม่ต้องใช้ seq_len
            if self.model_type == 'LINEAR':
                if train_len <= 0 or test_len <= 0:
                    print(f"[Adaptive Fold {i+1}] Skipping (train/test < 1): train({train_len}), test({test_len})")
                    continue
            else: # เงื่อนไขสำหรับ RNN/LSTM/GRU
                if train_len <= seq_len or test_len <= seq_len:
                    print(f"[Adaptive Fold {i+1}] Skipping (train/test < seq_len): train({train_len}), test({test_len}), seq_len({seq_len})")
                    continue

            split_point = int(fold_length * (1 - test_ratio))
            train_start = start
            train_end = start + split_point
            test_start = train_end
            test_end = end

            train_len = train_end - train_start
            test_len = test_end - test_start

            # ตรวจสอบ train/test ต้องมีขนาดมากกว่า sequence_length
            if train_len <= seq_len or test_len <= seq_len:
                print(f"[Adaptive Fold {i+1}] Skipping (train/test < seq_len): train({train_len}), test({test_len}), seq_len({seq_len})")
                continue

            X_train, X_test = X.iloc[train_start:train_end], X.iloc[test_start:test_end]
            y_train, y_test = y.iloc[train_start:train_end], y.iloc[test_start:test_end]

            rmse, mae = np.nan, np.nan
            if self.model_type in ['RNN', 'LSTM', 'GRU']:
                model = RNNRegressor(model_type=self.model_type, **self.model_params)
            elif self.model_type == 'LINEAR':
                model = LinearRegressionModel(**self.model_params)
            else:
                raise ValueError(f"Unknown model type: {self.model_type}")

            try:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                if self.model_type in ['RNN', 'LSTM', 'GRU']:
                    y_test_aligned = y_test.iloc[model.seq_generator.sequence_length:]
                    y_pred = y_pred[:len(y_test_aligned)]
                else:
                    y_test_aligned = y_test
                if len(y_pred) > 0 and len(y_test_aligned) > 0:
                    rmse = np.sqrt(mean_squared_error(y_test_aligned, y_pred))
                    mae = mean_absolute_error(y_test_aligned, y_pred)
                    metrics_rmse.append(rmse)
                    metrics_mae.append(mae)
                    print(f"[Adaptive Fold {i+1}] RMSE={rmse:.3f}, MAE={mae:.3f}")
                else:
                    print(f"[Adaptive Fold {i+1}] Not enough data to calculate metrics.")
            except Exception as e:
                print(f"[Adaptive Fold {i+1}] Error: {e}")

        return metrics_rmse, metrics_mae

class BaselineTimeSeriesCV:
    """
    Performs cross-validation for time series data using a rolling window.
    The data is split into n_splits + 1 parts.
    """
    def __init__(self, model_type: str = 'LSTM', model_params: dict = None,
                 n_splits: int = 4):
        self.model_type = model_type.upper()
        self.model_params = model_params or {}
        self.n_splits = n_splits
        if self.n_splits < 1:
            raise ValueError("n_splits must be at least 1.")

    def run(self, X: pd.DataFrame, y: pd.Series) -> Tuple[List[float], List[float]]:
        metrics_rmse, metrics_mae = [], []
        total_size = len(X)
        
        # Calculate the size of each part. The data is split into n_splits + 1 parts.
        part_size = total_size // (self.n_splits + 1)
        test_ratio = 0.2
        
        if part_size <= 0:
            raise ValueError("Not enough data for the specified number of splits.")

        for i in range(self.n_splits):
            # กำหนดช่วงข้อมูลสำหรับ Fold ปัจจุบัน (ข้อมูลส่วนที่ i+1)
            fold_start = i * part_size
            fold_end = (i + 1) * part_size
            if i == self.n_splits - 1: # จัดการส่วนสุดท้ายที่อาจมีขนาดไม่เท่ากัน
                fold_end = total_size

            fold_data_X = X.iloc[fold_start:fold_end]
            fold_data_y = y.iloc[fold_start:fold_end]

            # แบ่งชุดข้อมูลภายใน Fold เป็น Train และ Test
            split_point = int(len(fold_data_X) * (1 - test_ratio))
            
            X_train = fold_data_X.iloc[:split_point]
            y_train = fold_data_y.iloc[:split_point]
            
            X_test = fold_data_X.iloc[split_point:]
            y_test = fold_data_y.iloc[split_point:]

            # Check for sufficient data size for RNN-based models
            seq_len = self.model_params.get('sequence_length', 30) if self.model_type != 'LINEAR' else 0
            train_len = len(X_train)
            test_len = len(X_test)
            
            if train_len <= seq_len or test_len <= seq_len:
                print(f"[Baseline Fold {i+1}] Skipping (train/test < seq_len): train({train_len}), test({test_len}), seq_len({seq_len})")
                continue

            rmse, mae = np.nan, np.nan
            try:
                # Model instantiation and fitting
                if self.model_type in ['RNN', 'LSTM', 'GRU']:
                    model = RNNRegressor(model_type=self.model_type, **self.model_params)
                elif self.model_type == 'LINEAR':
                    model = LinearRegressionModel(**self.model_params)
                else:
                    raise ValueError(f"Unknown model type: {self.model_type}")

                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                # Align prediction and test data length for RNN models
                if self.model_type in ['RNN', 'LSTM', 'GRU']:
                    y_test_aligned = y_test.iloc[seq_len:]
                else:
                    y_test_aligned = y_test
                
                min_len = min(len(y_pred), len(y_test_aligned))
                if min_len > 0:
                    y_pred_trimmed = y_pred[:min_len]
                    y_test_trimmed = y_test_aligned[:min_len]
                    rmse = np.sqrt(mean_squared_error(y_test_trimmed, y_pred_trimmed))
                    mae = mean_absolute_error(y_test_trimmed, y_pred_trimmed)
                    metrics_rmse.append(rmse)
                    metrics_mae.append(mae)
                    print(f"[Baseline Fold {i+1}] RMSE={rmse:.3f}, MAE={mae:.3f}")
                else:
                    print(f"[Baseline Fold {i+1}] Not enough data to calculate metrics.")
            except Exception as e:
                print(f"[Baseline Fold {i+1}] Error during model training/prediction: {e}")
                continue
                
        return metrics_rmse, metrics_mae
    
# --- 5. Model Comparison and Execution ---

class ModelComparison:
    """Compares the performance of different models using two CV strategies."""
    def __init__(self, rnn_params: dict = None, linear_params: dict = None):
        self.rnn_params = rnn_params or {}
        self.linear_params = linear_params or {}
        self.models = ['RNN', 'LSTM', 'GRU', 'LINEAR']
        
    def compare_models(self, X: pd.DataFrame, y: pd.Series, drift_points: List[int]):
        results = {}

        for model_type in self.models:
            params = self.linear_params if model_type == 'LINEAR' else self.rnn_params
            print(f"\n{'='*50}")
            print(f"Testing {model_type} Model")
            print(f"{'='*50}")
            # Adaptive CV ใช้จุด drift ทั้งหมดที่ตรวจพบ
            drift_cv = DriftAdaptiveTimeSeriesCV(model_type, params)
            drift_rmse, drift_mae = drift_cv.run(X, y, drift_points)
            
            # Baseline CV ใช้การแบ่ง 5-fold แบบปกติ
            baseline_cv = BaselineTimeSeriesCV(model_type, params, n_splits=5)
            base_rmse, base_mae = baseline_cv.run(X, y)
            
            results[model_type] = {'adaptive_rmse': drift_rmse, 'adaptive_mae': drift_mae, 'baseline_rmse': base_rmse, 'baseline_mae': base_mae}
        return results
    
    def print_summary(self, results: dict):
        print("\n" + "="*80)
        print("MODEL COMPARISON SUMMARY")
        print("="*80)
        for model_type in self.models:
            if model_type in results:
                print(f"\n{model_type} Results:")
                print("-" * 30)
                if results[model_type]['adaptive_rmse']:
                    avg_rmse = np.mean(results[model_type]['adaptive_rmse'])
                    avg_mae = np.mean(results[model_type]['adaptive_mae'])
                    print(f"Adaptive CV - Avg RMSE: {avg_rmse:.3f}, Avg MAE: {avg_mae:.3f} ")
                else:
                    print("Adaptive CV - No valid results")
                if results[model_type]['baseline_rmse']:
                    avg_rmse = np.mean(results[model_type]['baseline_rmse'])
                    avg_mae = np.mean(results[model_type]['baseline_mae'])
                    print(f"Baseline CV - Avg RMSE: {avg_rmse:.3f}, Avg MAE: {avg_mae:.3f}")
                else:
                    print("Baseline CV - No valid results")
        best_model = self._find_best_model(results)
        if best_model:
            print(f"\nBest Model: {best_model}")
    
    def _find_best_model(self, results: dict):
        best_model, best_score = None, float('inf')
        for model_type in self.models:
            if model_type in results:
                scores = results[model_type]['adaptive_rmse'] or results[model_type]['baseline_rmse']
                if scores and np.mean(scores) < best_score:
                    best_score = np.mean(scores)
                    best_model = model_type
        return best_model

# --- Main Execution Block ---
if __name__ == "__main__":
    detector = ADWINDriftDetector(delta=0.01, min_fold_len=15)
    drift_points = detector.detect(df, 'Close')
    drift_dates_formatted = df.iloc[drift_points]['Date'].dt.strftime('%d/%m/%Y').tolist()
    print("\nDrift Dates:")
    print(drift_points)
    print(drift_dates_formatted)
    print("Len :", len(drift_dates_formatted))
    
    # แก้ไข parameters ของ RNN
    
    rnn_params = {'sequence_length': 15, 'units': 32, 'dropout_rate': 0.2, 'learning_rate': 0.001, 'epochs': 50, 'batch_size': 32, 'verbose': 0}
    linear_params = {'fit_intercept': True}
    comparator = ModelComparison(rnn_params=rnn_params, linear_params=linear_params)
    results = comparator.compare_models(X, y, drift_points)
    comparator.print_summary(results)

# Show Drift

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# ใช้ข้อมูล Date กับ Close price สำหรับกราฟหลัก
dates = df['Date']
prices = df['Close']

# Map drift point index เป็นวันที่
drift_dates = df.iloc[drift_points]['Date'].values

# Plot กราฟราคาพร้อมเส้นแสดง drift point
plt.figure(figsize=(14, 6))
sns.lineplot(x=dates, y=prices, label='Close Price', color='blue')

# วาดเส้นแนวตั้งสำหรับแต่ละ drift point
for d in drift_dates:
    plt.axvline(x=d, color='red', linestyle='-', alpha=0.7)

# ตกแต่งกราฟ
plt.title("Drift Points Detected in META Closing Prices")
plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()



# แปลง drift point index เป็นวันที่แบบ วัน/เดือน/ปี
drift_dates_formatted = df.iloc[drift_points]['Date'].dt.strftime('%d/%m/%Y').tolist()
print("Drift Dates of META :")
print(drift_dates_formatted)
